This Section Does the database creation; Probably only ever needs to be run once:

In [2]:
import couchdb

server_url = "http://localhost:5984/"
new_db_name = "gdbscrapedresultsfulltext"
remote_server = couchdb.Server(server_url)

db = remote_server.create(new_db_name)


This section does the bulk of the processing.

In [3]:
#Test Section
import couchdb

remote_server = couchdb.Server(url=server_url)
write_db_name = "gdbscrapedresultsfulltext"
write_db = remote_server[write_db_name]

loc = write_db.find({
                "selector": {
                  "record_number": {
                    "$eq": "12345"
                  }
                }
              })



This is the main code to pull data:

In [ ]:
import couchdb

from TextScraper import FullTextScraper
from lxml.etree import SerialisationError

# FullTextScraper.textgetter()
server_url = "http://localhost:5984/"
write_db_name = "gdbscrapedresultsfulltext"

remote_server = couchdb.Server(url=server_url)
db = remote_server['gdbscraperdb']
write_db = remote_server[write_db_name]

counter = 0
for row in db.find({"selector": {"$is_junk" : { "$ne": True } }} ):
    # print(row["url"])
    url_ = row["url"]
    url_ = url_.strip()
    url_ = url_[1:]
    url_ = url_[:-1]
    record_number = row['record_number']
    loc = write_db.find({
                    "selector": {
                      "record_number": {
                        "$eq": record_number
                      }
                    }
                  })
    #This will continue if we have already pulled this record's url
    if len(list(loc)) != 0:
        continue
    
    try:
        ret = FullTextScraper.textgetter(url_)
    except Exception as e:
        print("Serialization Error, continuing" + url_)
        d = {
                        "record_number":record_number,
                        "is_error":"true"
                    }
        write_db.save(d)
        write_db.commit()
        continue
        
    # print(ret)
    try:
        article_title = ret["title"]
    except TypeError as e:
        continue
    article_url = url_
    article_keywords = ret['keywords']
    article_summary = ret['summary']
    article_text = ret['text']
    
    d = {
                "record_number":record_number,
                "title":article_title,
                "url":url_,
                "keywords":article_keywords,
                "summary":article_summary,
                "text":article_text
            }
    try:
        write_db.save(d)
    except Exception as e:
        print("some exception")
        print(d)
    write_db.commit()

